<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/10_finetuning_BERT_for_classification_(in_class).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 感情分析のための日本語BERTのfinetuning

* Transformersについては、例えば、下記のWebページを参照。
 * http://nlp.seas.harvard.edu/2018/04/03/attention.html

* 今回は、Hugging FaceのTransformersを使う。
 * https://huggingface.co/docs/transformers/

* ランタイムのタイプはGPUにしておく。

## transformersのインストール

In [1]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 41.3 MB/s 
     |████████████████████████████████| 7.6 MB 58.8 MB/s 
     |████████████████████████████████| 182 kB 56.9 MB/s 


### トークナイザを動かすために必要なモジュールのインストール

In [2]:
!pip install fugashi ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 615 kB 29.3 MB/s 
     |████████████████████████████████| 13.4 MB 67.6 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=a0846e2bee02eebf6eb0f9ee2493776d044eb88233aa5924928553bdd968ddfe
  Stored in directory: /root/.cache/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
Successfully built ipadic


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 日本語BERTとしては、今回、下記のモデルを使う。
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"

## データセット

### WRIME: 主観と客観の感情分析データセット
* https://github.com/ids-cv/wrime

* このデータセットは、元々は-2, -1, 0, ,1 2の５値で感情極性を表現している。
* 今回は、**ネガティブ、ニュートラル、ポジティブの3値**に単純化することにする。
 * 余裕がある方は、5値分類のままファインチューニングを実施してみてください。

In [4]:
!wget https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver2.tsv

--2022-12-17 02:04:34--  https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver2.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8182156 (7.8M) [text/plain]
Saving to: ‘wrime-ver2.tsv’

wrime-ver2.tsv      100%[===================>]   7.80M  --.-KB/s    in 0.03s   

2022-12-17 02:04:34 (278 MB/s) - ‘wrime-ver2.tsv’ saved [8182156/8182156]



In [5]:
df = pd.read_csv("wrime-ver2.tsv", sep="\t")
df.head()

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Sentiment,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust,Avg. Readers_Sentiment
0,ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…,1,2012/7/31 23:48,train,0,1,2,1,1,0,...,-1,0,2,0,0,0,0,0,0,-1
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな〜 と立ち止まる帰り道。チャリなし生活も...,1,2012/8/2 23:09,train,3,0,3,0,0,0,...,1,1,0,0,2,0,0,0,0,1
2,早寝するつもりが飲み物がなくなりコンビニへ。ん、今日、風が涼しいな。,1,2012/8/5 0:50,train,1,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,0
3,眠い、眠れない。,1,2012/8/8 1:36,train,0,2,1,0,0,1,...,-1,0,1,0,0,0,0,1,0,-1
4,ただいま〜 って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,1,2012/8/9 22:24,train,2,1,3,2,0,1,...,0,1,0,0,1,0,0,0,0,0


* データセットはすでにTrain/Dev/Testの3つの部分に分けられている。

In [6]:
split_tags = list(df["Train/Dev/Test"].unique())
split_tags

['train', 'test', 'dev']

* 今回は、ポジティヴ／ニュートラル／ネガティヴの3値分類問題として、感情分析を行なう。

In [7]:
df_sentiment = df[['Train/Dev/Test', 'Avg. Readers_Sentiment', 'Sentence']]

* ポジティブのラベルを2、ネガティブのラベルを1、ニュートラルのラベルを0と設定する。

In [8]:
df_sentiment.loc[df['Avg. Readers_Sentiment'] > 0, 'Target'] = 2
df_sentiment.loc[df['Avg. Readers_Sentiment'] < 0, 'Target'] = 1
df_sentiment.loc[df['Avg. Readers_Sentiment'] == 0, 'Target'] = 0

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


* 各クラスのテキスト数を調べる。

In [9]:
print(f"{(df_sentiment['Target'] == 2).sum()} positive texts")
print(f"{(df_sentiment['Target'] == 1).sum()} negative texts")
print(f"{(df_sentiment['Target'] == 0).sum()} neutral texts")

11383 positive texts
12155 negative texts
11462 neutral texts


* クラス数を変数`num_class`に保存しておく。

In [10]:
num_class = len(df_sentiment["Target"].unique())
print(num_class)

3


* データセットで元々指定されたtrain/dev/testの分割のまま、データセットを分割する。

In [11]:
df_splits = {}
for split_tag in split_tags:
  df_splits[split_tag] = df_sentiment[df_sentiment['Train/Dev/Test'] == split_tag][['Target', 'Sentence']]

In [12]:
df_splits["train"].head()

,Target,Sentence
0,1.0,ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…
1,2.0,今日の月も白くて明るい。昨日より雲が少なくてキレイな〜 と立ち止まる帰り道。チャリなし生活も...
2,0.0,早寝するつもりが飲み物がなくなりコンビニへ。ん、今日、風が涼しいな。
3,1.0,眠い、眠れない。
4,0.0,ただいま〜 って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!


## Dataset

* データフレームからPyTorchのDatasetを作る。

In [13]:
class MyDataset(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    return self.df.iloc[idx]['Sentence'], self.df.iloc[idx]['Target']

In [14]:
wrime = {}
for split_tag in split_tags:
  wrime[split_tag] = MyDataset(df_splits[split_tag])

In [15]:
wrime["train"][0]

('ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…', 1.0)

## トークナイザ

* 日本語BERTのトークナイザをダウンロードしておく。

In [16]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [17]:
tokenizer

PreTrainedTokenizer(name_or_path='cl-tohoku/bert-base-japanese-whole-word-masking', vocab_size=32000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

* 試しに、一つのテキストをトークナイズしてみる。

In [18]:
model_inputs = tokenizer(wrime["train"][0][0], padding=True, return_tensors="pt")
print(model_inputs)

{'input_ids': tensor([[    2,  3937, 28517, 26099,    15,    16,  3318, 12272,   640,     8,
          1131, 28479,    31,    40,  2949,     7,    12,  1549,  5602,  3215,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


* `token_type_ids`については、下記URLを参照。
 * https://huggingface.co/docs/transformers/main/en/glossary#token-type-ids

* インデックスの列をトークン列に戻してみる。
 * 先頭と末尾のspecial tokensに注意。

In [19]:
print(tokenizer.convert_ids_to_tokens(model_inputs['input_ids'][0]))

['[CLS]', 'ぼ', '##け', 'っと', 'し', 'て', 'たら', 'こんな', '時間', '。', 'チャ', '##リ', 'ある', 'から', '食べ', 'に', 'で', 'たい', 'のに', '...', '[SEP]']


## DataLoader

* PyTorchのDataLoaderを利用して、ミニバッチをランダムな順で取ってこれるようにする。

* どんなミニバッチにしたいかをcollation用関数で定義する。
 * テキストはトークナイザを通した状態でミニバッチに含ませる。
 * ラベルはPyTorchのTensorに変換してミニバッチに含ませる。

* 今回使っているトークナイザは、テキストの長さが揃っていないときのpaddingの処理までおこなってくれる。

In [20]:
def collate_fn(batch):
  texts, labels = zip(*batch)
  model_inputs = tokenizer(texts, padding=True, return_tensors="pt")
  return model_inputs.to(device), torch.tensor(labels).long().to(device)

* Train/Dev/Testの各スプリットについて、DataLoaderのインスタンスを作成する。
 * ミニバッチのサイズは、チューニングした方が良い。

In [21]:
BATCH_SIZE = 8
loaders = {}
for split_tag in split_tags:
  loaders[split_tag] = DataLoader(
      wrime[split_tag],
      batch_size=BATCH_SIZE,
      shuffle=True,
      collate_fn=collate_fn,
      )

* 試しに、一個、ミニバッチを訓練データから取得してみる。

In [22]:
input, label = next(iter(loaders["train"]))
print(input)
print(label)

{'input_ids': tensor([[    2,   237, 28737,     7,   322,    10,    54,     8,    53,    14,
            21,    80,   679,     3,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [    2,  1350, 28564, 11913, 28478, 28561,     7,   517,    16,   322,
          3913,    10,   679, 14993, 28504, 16184,    10,    18, 28474,  3215,
         29068,  2198,    12,   234,  7796,    16,     6,  1350, 28564, 28564,
             7

## 事前学習済み日本語BERT

### 事前学習済みモデルのダウンロード

In [23]:
bert = AutoModel.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

* このBERTの単語埋め込みの次元数を調べる。

In [26]:
bert.embeddings.word_embeddings.weight.shape

torch.Size([32000, 768])

* このBERTのlayer数を調べる。

In [27]:
len(bert.encoder.layer)

12

* 最後のlayerだけ表示する。

In [28]:
bert.encoder.layer[-1]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

### pooler
* poolerは、最初のトークンに対応する出力を受け取っている。
 * https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L652
* 活性化関数tanhを使っている。

In [29]:
bert.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

## 分類モデルの定義

In [30]:
class BERTTextSentiment(nn.Module):
  def __init__(self, bert, num_class):
    super().__init__()
    self.bert = bert
    self.embdim = bert.embeddings.word_embeddings.weight.size(1)
    self.num_class = num_class
    self.fc = nn.Linear(self.embdim * 2, self.num_class)

  def forward(self, input):
    output = self.bert(**input)
    pooler_output = output.pooler_output # [CLS]に対応するhidden state
    # poolerに合わせて、tanhを適用しておく。
    # last_hidden_state (batch size, seq len, hidden dim)
    mean_output = torch.tanh(output.last_hidden_state).mean(1)
    logit = self.fc(torch.cat([pooler_output, mean_output], -1))
    return logit

In [31]:
model = BERTTextSentiment(bert, num_class).to(device)

* 最後のlayerと、poolerだけfinetuningするよう、設定する。

In [32]:
for param in model.bert.parameters():
  param.requires_grad = False
for param in model.bert.encoder.layer[-1].parameters():
  param.requires_grad = True
for param in model.bert.pooler.parameters():
  param.requires_grad = True

* 試しに、ミニバッチを入力してみる。

In [33]:
input, _ = next(iter(loaders["train"]))
logit = model(input)
print(logit)

tensor([[-0.2432,  0.1917, -0.2481],
        [-0.2075,  0.2245, -0.1854],
        [-0.1966,  0.2440, -0.1939],
        [-0.1093,  0.2181, -0.1143],
        [-0.1018,  0.4097, -0.3131],
        [-0.3380,  0.3761, -0.3100],
        [-0.2255,  0.2822, -0.0690],
        [-0.4497,  0.4179, -0.2122]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


## 最適化アルゴリズムと損失関数

* 学習率はチューニングする必要あり。

In [34]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

## 訓練を行なう関数

In [36]:
def train(dataloader):
  model.train()
  total_loss, total_acc, total_count = 0, 0, 0
  for i, batch in enumerate(dataloader):
    input, label = batch
    logit = model(input)
    loss = criterion(logit, label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    n_instances = label.size(0)
    total_loss += loss.item() * n_instances
    total_acc += (logit.argmax(-1) == label).sum().item()
    total_count += n_instances
    if (i + 1) % 100 == 0:
      print(f"===>{i+1} | acc {total_acc/total_count:.4f}")
  return total_loss / total_count, total_acc / total_count


## 評価を行なう関数

In [37]:
def evaluation(dataloader):
  model.eval()
  total_loss, total_acc, total_count = 0, 0, 0
  for i, batch in enumerate(dataloader):
    input, label = batch
    with torch.no_grad():
      logit = model(input)
      loss = criterion(logit, label)
    n_instances = label.size(0)
    total_loss += loss.item() * n_instances
    total_acc += (logit.argmax(-1) == label).sum().item()
    total_count += n_instances
    if (i + 1) % 100 == 0:
      print(f"===>{i+1} | acc {total_acc/total_count:.4f}")
  return total_loss / total_count, total_acc / total_count

## finetuningの実行

In [38]:
for epoch in range(1, 6):
  loss, acc = train(loaders["train"])
  dev_loss, dev_acc = evaluation(loaders["dev"])
  print(f'> epoch {epoch} | train loss {loss:.3f} | train acc {acc:.4f} || '
      f'dev loss {dev_loss:.3f} | dev acc {dev_acc:.4f}')

===>100 | acc 0.4012
===>200 | acc 0.4400
===>300 | acc 0.4817
===>400 | acc 0.5106
===>500 | acc 0.5250
===>600 | acc 0.5315
===>700 | acc 0.5398
===>800 | acc 0.5498
===>900 | acc 0.5582
===>1000 | acc 0.5627
===>1100 | acc 0.5710
===>1200 | acc 0.5752
===>1300 | acc 0.5787
===>1400 | acc 0.5816
===>1500 | acc 0.5833
===>1600 | acc 0.5870
===>1700 | acc 0.5921
===>1800 | acc 0.5959
===>1900 | acc 0.5966
===>2000 | acc 0.5968
===>2100 | acc 0.5983
===>2200 | acc 0.5995
===>2300 | acc 0.6007
===>2400 | acc 0.6027
===>2500 | acc 0.6031
===>2600 | acc 0.6043
===>2700 | acc 0.6043
===>2800 | acc 0.6045
===>2900 | acc 0.6060
===>3000 | acc 0.6070
===>3100 | acc 0.6083


KeyboardInterrupt: ignored

# 課題7
* 分類性能が上がるよう、ハイパーパラメータをチューニングしてみよう。
 * 学習率、ミニバッチのサイズ、分類用の全結合層の層数、etc...
* [発展] 今回のデータセットの、元々の5値分類で、finetuningを実行してみよう。